In [21]:
from cases import *

# case = Debugger3
cases = [case_1, case_2, case_3, case_4, case_5]


# Simulated Annealing

Here we first use a new random seed to guaranty that our random generator acts differently after each run.

## Problem Representation

Being inspired by the Genetic Algorithms, we represent each solution to our program by a gene, that is, each solution is a permutation of the orders, showing the order of the orders that should be processed. Therefore, our fitness function is calculated as described:

We iterate through the orders, and if we cannot process that order with the current log, we obtain a new log (increasing the fitness value). To optimize our solution, we just made a modification in how we process the orders. In other words, we keep track of wastes, so that if the next order can be processed using that wastes, we let it be.

The rest is just standard implementation of the algorithm according to the https://link.springer.com/article/10.1186/2251-712X-8-24 article.

In [22]:
import numpy as np
from time import time 
import math

np.random.seed(int(time()))
EPS = 1

class Gene:
    fitness = 0
    def __init__(self, perm) -> None:
        self.perm = perm

class SA:
    def __init__(self, case: Case, T, alpha, mcn, mgni) -> None:
        self.case = case
        self.state = Gene(case.req.copy())
        np.random.shuffle(self.state.perm)
        self.state.fitness = self.calculate_fitness(self.state)
        self.fittest = self.state

        self.T = T
        self.alpha = alpha
        self.N = mcn
        self.n = 0

        self.gen = 0
        self.mgni = mgni # Max Generations Without Improvement
        self.terminated = False

    def iterate(self):
        if self.gen == self.mgni:
            self.terminated = True
            return

        if self.n == self.N:
            self.T = self.new_T(self.T)
            self.n = 0

        self.n += 1
        self.gen += 1

        neighbour = self.get_random_neighbour(self.state)
        fitness = self.calculate_fitness(neighbour)
        
        neighbour.fitness = fitness
        delta = fitness - self.state.fitness

        if delta < 0:
            self.change_state(neighbour)
            return
        
        prob = self.prob_func(delta, self.T)
        unif = np.random.random()

        if unif <= prob:
            self.change_state(neighbour)

    def calculate_fitness(self, state):
        L = self.case.L
        result = 0
        curr_log = 0
        loss = {}

        for order in state.perm:
            if order > curr_log:
                if curr_log not in loss:
                    loss[curr_log] = 0
                loss[curr_log] += 1
                curr_log = L
                result += 1
            mn = min([1_000_000_000] + [i for i in loss if i >= order])
            if mn == 1_000_000_000 or loss[mn] <= 0:
                curr_log -= order
            else:
                loss[mn] -= 1
                new_loss = mn - order
                if new_loss not in loss:
                    loss[new_loss] = 0

                loss[new_loss] += 1

        if curr_log == L:
            result -= 1

        return result

    def get_random_neighbour(self, state: Gene):
        pos1, pos2 = 0, 0
        while pos1 == pos2:
            pos1 = np.random.randint(0, len(state.perm))
            pos2 = np.random.randint(0, len(state.perm))

        result = Gene(state.perm.copy())
        result.perm[pos1], result.perm[pos2] = result.perm[pos2], result.perm[pos1]

        return result

    def change_state(self, new_state):
        if new_state.fitness - self.state.fitness <= EPS:
            self.gen = 0
            self.fittest = new_state
        self.state = new_state

    def new_T(self, old_T):
        return self.alpha * old_T

    def prob_func(self, d, T):
        return math.pow(math.e, -float(d)/float(T))


In [28]:
T = 100000
alpha = 0.95
markov_chain_length = 150
instances = 20
problems = {}

for case in cases:
    for instance in range(instances):
        if case.name not in problems:
            problems[case.name] = []

        problem = SA(case, T, alpha, markov_chain_length, 20 * markov_chain_length)
        problems[case.name].append(problem)

done = False

while not done:
    flag = True
    for key in problems:
        for problem in problems[key]:
            problem.iterate()
            flag = flag & problem.terminated

    if not flag:
        done = True


In [30]:
best_solutions = {}

for case in problems:
    for problem in problems[case]:
        if case not in best_solutions:
            best_solutions[case] = problem.fittest
        
        if best_solutions[case].fitness > problem.fittest.fitness:
            best_solutions[case] = problem.fittest
    

In [31]:
for case in best_solutions:
    print(f"best answer found for {case} is {best_solutions[case].fitness}")

best answer found for case 1 is 52
best answer found for case 2 is 80
best answer found for case 3 is 97
best answer found for case 4 is 209
best answer found for case 5 is 3900


In [33]:
for case in best_solutions:
    print(f"best answer found for {case} is {best_solutions[case].perm}")
    print("\n-------------------------------\n")

best answer found for case 1 is [171, 557, 495, 517, 278, 149, 266, 75, 315, 507, 118, 107, 86, 109, 370, 689, 653, 186, 988, 412, 266, 557, 368, 716, 246, 149, 170, 80, 967, 788, 268, 125, 518, 409, 125, 457, 753, 627, 295, 286, 402, 224, 145, 987, 78, 181, 123, 88, 312, 868, 333, 33, 544, 116, 46, 609, 180, 230, 232, 264, 135, 414, 149, 437, 71, 251, 61, 356, 463, 632, 424, 144, 549, 106, 117, 53, 106, 218, 753, 106, 18, 280, 506, 532, 354, 92, 662, 241, 148, 301, 346, 92, 592, 914, 99, 501, 365, 306, 686, 249, 283, 351, 106, 555, 43, 371, 84, 187, 581, 672, 70, 557, 678, 648, 211, 292, 45, 441, 660, 525, 460, 248, 286, 115, 69, 284, 88, 515, 312, 405, 23, 337, 788, 126, 119, 618, 933, 805, 79, 60]

-------------------------------

best answer found for case 2 is [1710, 2000, 1880, 2150, 1880, 2200, 1930, 1820, 1930, 1380, 1820, 1380, 1380, 1560, 2200, 1820, 2200, 1710, 1380, 1820, 1560, 2140, 2140, 2100, 2200, 1820, 1710, 1710, 2200, 2100, 1820, 2100, 1520, 1880, 2150, 2140, 2100, 1